## Advanced Prompt Engineering Concepts

In this notebook, you will learn the concepts of:
* Auto-Prompting
* Prompt Chaining

To explore how these concepts can be brought into real world scenarios, we will be simulating how it can be used to write a report for us. The report will be written with analysis the LLM derived based on factual information. Here is the workflow of how it works:
1. Set a Report Title to be written.
2. Make use of auto-prompting to generate questions that are important to help derive an analysis on the given report title.
3. Based on the generated questions, search on internal documents for the information.
4. Using the extracted information, write the report.

This can be useful where you want to generate a certain report. Auto prompting helps to figure out what supporting information will be useful in writing the report and search for the answers. The search can be online search, internal document search etc. Then, the report is written based on the facts.

In [6]:
import logging
import json
import time
import boto3

import matplotlib.pyplot as plt
import numpy as np
import requests
from tqdm.contrib.concurrent import thread_map

region = boto3.Session().region_name

logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

url = "https://e165qoav65.execute-api.us-east-1.amazonaws.com/test/"

In [118]:
# Define a method that will invoke the REST API via HTTP request to interact with the model. The returned response will be the
# text generated by the model.
def query_endpoint_with_json_payload(url, payload):
    response = requests.post(
        url,
        json=payload,
    )
    
    response_body = response.content
    response_json = json.loads(response_body.decode('utf-8'))
    generated_text = response_json['completions'][0]['data']['text']
    return generated_text

### Auto Prompt Generation
This section simulates how LLM can automatically generate questions related to a certain topic. In this simulation, the question generations are closely guided by the input prompt. With domain adaptation fine-tuning or extensive prompt engineering, the LLM will be able to achieve quality results with more generic prompts.

This concept explores how LLM can be used to dissect a problem, explore further analysis to be made that can be helpful to the given problem.

In [137]:
report = "2023 financial outlook for United States"
generate_analysis_prompt = f'''Financial outlook for a country are typically dependent on the following factors - Consumer spending, foreign trade.
Act as an expert financial analyst. Based on each of the factors, generate a detailed question that can draw insights for {report}. Example:
1. Question 1?
2. Question 2?

Questions:
'''

logger.info(f"\u001b[4mPrompt\u001b[0m\n{generate_analysis_prompt}")

Prompt
Financial outlook for a country are typically dependent on the following factors - Consumer spending, foreign trade.
Act as an expert financial analyst. Based on each of the factors, generate a detailed question that can draw insights for 2023 financial outlook for United States. Example:
1. Question 1?
2. Question 2?

Questions:



Typically, a more generic prompt like this will be more helpful. However, due to the limited 'documents' available in this simulation, it is scoped down to consumer spending and foreign trade.

In [110]:
generate_analysis_questions_sample = f'''Act as an expert financial analyst. Generate 10 detailed questions that can draw insights for {report}. Example:
1. Question 1?
2. Question 2?

Questions:
'''

logger.info(f"\u001b[4mPrompt\u001b[0m\n{generate_analysis_questions_sample}")

payload = {"prompt": generate_analysis_questions_sample, "maxTokens": 200, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)

logger.info(f'{response}')

Prompt
Act as an expert financial analyst. Generate 10 detailed questions that can draw insights for 2023 financial outlook for United States. Example:
1. Question 1?
2. Question 2?

Questions:

1. What are the key economic indicators that will drive the US economic outlook in 2023?
2. How will the US Federal Reserve's monetary policy decisions impact the economy in 2023?
3. How will the US housing market perform in 2023?
4. How will the US labor market perform in 2023?
5. How will the US stock market perform in 2023?
6. How will the US budget deficit and national debt be impacted by policy decisions in 2023?
7. How will the US trade policy impact the economy in 2023?
8. How will the US fiscal policy impact the economy in 2023?
9. How will the US healthcare system impact the economy in 2023?
10. How will the US energy sector impact the economy in 2023?


In [120]:
payload = {"prompt": generate_analysis_questions, "maxTokens": 200, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)

logger.info(f'{response}')

1. How will consumer spending in the United States affect the country's financial outlook in 2023?
2. How will foreign trade with the United States affect the country's financial outlook in 2023?


### Retrieving information from the stored documents
This section simulates how LLM can generate insights based on a provided context retrieved from internal documents. Tihs financial analysis is taken from https://www2.deloitte.com/us/en/insights/economy/us-economic-forecast/united-states-outlook-analysis.html.

In [134]:
consumer_spending = """Consumer spending
The near-term outlook for consumer spending turns on two big questions:

1. What will happen when consumers finish running down their pandemic savings?

In 2020, during the height of the pandemic, we estimated that households saved about US$1.6 trillion more than we forecasted before the pandemic. Most of that money has been spent, as the savings rate has dropped from an average of around 9% before the pandemic to around 3% in the final quarter of 2022. Many households still have more cash on hand now than they normally would want, but how much of that will they spend as the economy slows? One possibility is that many consumers will remain cautious and hold on to those savings even as they are able to go out and spend. Another possibility: Spending booms for a while longer as “revenge spending” on travel and consumer services continue to grow post pandemic. The baseline Deloitte forecast assumes that continued job and income growth will support continued growth in consumer spending, but spending will slow as the savings rate eventually rises back to the 6% range.

2. As consumer services recover, what happens to durable goods?

The pandemic sparked a remarkable change in consumer spending patterns. Spending on durable consumer goods jumped US$136 billion in 2020, while spending on services fell US$473 billion over the same period. Households substituted bicycles, gym equipment, and electronics for restaurants, entertainment, and travel. Once households can again purchase services, will they begin buying fewer goods? That may be happening, as, by Q4 2022, durables spending was down 11% from the peak in Q2 2021. In Q4 2022, durable goods accounted for 12% of total consumer spending, up from 10.5% in 2019. If consumers just return to their prepandemic spending patterns, durable consumer goods sellers will be looking at a 20% fall in spending. And consumers could conceivably spend even less since the durable goods they previously bought aren’t going to wear out that quickly.

Deloitte’s forecast assumes that consumer spending grows more slowly than income over the forecast horizon as households return to previous savings patterns. Durable goods spending continues to fall over the next few years as consumer spending “renormalizes” and consumers resume spending on services.

Over the longer term, we expect the pandemic to exacerbate some existing problems. It has thrown the problem of inequality into sharp relief, straining the budgets and living situations of millions of lower-income households. These are the very people who are less likely to have health insurance—especially after layoffs—and more likely to have health conditions that complicate recovery from infection. And retirement remains a significant issue: Even before the crisis, fewer than four in 10 nonretired adults described their retirement as on track, with a quarter of nonretired adults saying they had no retirement savings. The stock market boom will have little impact on many people’s balance sheets, leaving many people still unable to afford retirement as they age"""

In [135]:
consumer_spending_context = consumer_spending

consumer_spending_question = 'How will consumer spending trends impact the economy in 2023?'

consumer_spending_prompt = f'''Context:
{consumer_spending_context}

Act as an expert financial analyst. Use insights based on the given context, give an extremely detailed analysis for the following topic:
{consumer_spending_question}'''
logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Context:
Consumer spending
The near-term outlook for consumer spending turns on two big questions:

1. What will happen when consumers finish running down their pandemic savings?

In 2020, during the height of the pandemic, we estimated that households saved about US$1.6 trillion more than we forecasted before the pandemic. Most of that money has been spent, as the savings rate has dropped from an average of around 9% before the pandemic to around 3% in the final quarter of 2022. Many households still have more cash on hand now than they normally would want, but how much of that will they spend as the economy slows? One possibility is that many consumers will remain cautious and hold on to those savings even as they are able to go out and spend. Another possibility: Spending booms for a while longer as “revenge spending” on travel and consumer services continue to grow post pandemic. The baseline Deloitte forecast assumes that continued job and income growth will support continu

In [82]:
payload = {"prompt": prompt, "maxTokens": 500, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)

logger.info(f'{response}')


Consumer spending is a key driver of economic growth, accounting for about 70% of GDP in the United States. As the economy recovers from the COVID-19 pandemic, consumer spending is expected to play a key role in driving economic growth. However, consumer spending trends are expected to vary significantly in 2023, depending on several factors.

One key factor that will shape consumer spending trends in 2023 is the level of savings that households have accumulated during the pandemic. During the pandemic, households saved a significant amount of money, as spending on consumer services fell and spending on durable goods increased. As households begin to spend down their savings, consumer spending is expected to rise. However, the level of savings that households have accumulated and their willingness to spend it will determine the pace of consumer spending growth in 2023.

Another factor that will shape consumer spending trends in 2023 is the recovery of consumer services. As the economy

In [139]:
foreign_trade = '''
Foreign trade
The Russian invasion of Ukraine has created some headwinds for US exporters. Lower demand from Europe (market for 15% of US exports) and a higher dollar because of greater global risk have created some short-term challenges.

Beyond the Ukraine crisis, things look more positive. Real US exports are now slightly above the prepandemic level. As global financial and economic conditions normalize, US exporters are expected to see more opportunities. Demand for US goods is likely to rise in the medium term as the global economy recovers from the pandemic and the Ukraine invasion shock. Further, more normal financial conditions would create more opportunities for investment outside the United States and less desire to hold dollars to avoid risk. That will potentially lower the dollar, making US exports more competitive globally.

Meanwhile, real US imports have fallen in the last two quarters. There has been a substantial decline in imports of consumer goods, reflecting the shift in consumer spending from goods to services. That is likely to continue, helping to dampen import growth.

But there is another trend that is critical to consider. Over the past few years, many analysts have begun to expect a move toward deglobalization. Global exports grew from 13% of global GDP in 1970 to 34% in 2012. But since then, the share of exports in global GDP started to fall as globalization stalled, and opponents of freer trade started to gain more political influence. All this points to the potential unraveling of the policies that fostered the earlier globalization.

COVID-19 may have accelerated this shift. Although COVID-19 is a global phenomenon, leaders made major decisions about how to fight it—in both health and economic policy—on a country-by-country basis. Examples of this are the US withdrawal from cooperation in the World Health Organization in 2020 (although the United States has since rejoined) and the unilateral decisions of both China and Russia to deploy their own vaccines before the completion of phase 3 trials. And countries with vaccine-manufacturing facilities rushed to vaccinate their own citizens rather than cooperating on a global vaccination plan. All this was in stark contrast to the joint global approach that public health professionals might have recommended.

On top of this, the US-China trade conflict continues. The White House has shown some interest in returning to a multilateral approach to trade—for example, by supporting Ngozi Okonjo-Iweala for the World Trade Organization director-general. However, US Trade Representative Katherine Tai has made a point of stating that trade policy should be aimed at helping US workers. And many of the Trump-era tariffs remain in place, with little prospect that the tariffs on China, in particular, will be withdrawn.

In addition to this, many businesses are considering rebuilding their supply chains to create more resilience in the face of unexpected events such as the pandemic and changes in US trade policy. The imperative for such changes has become stronger with the increasing supply chain issues and port delays facing importers of key components and consumer goods. It’s impossible, of course, to simply and quickly refashion supply chains to reduce foreign dependence. American companies will continue to source from China in the coming years. But companies will likely accelerate attempts to reduce their dependence on China (a process they had begun before the pandemic). Building more robust supply chains may mean moving production back to the United States, or perhaps to Mexico or some other, closer source. Or it may mean a portfolio of suppliers rather than a single source—even if the single source is the cheapest.

Reengineering supply chains will inevitably mean a rise in overall costs. Just as the “China price” held inflation in check for years, an attempt to avoid dependency on China might create inflationary pressures in the later years of our forecast horizon. And if markets won’t accept inflation, companies may to have to accept lower profits to diversify supply chains. Globalization offered a comparatively painless way to improve many people’s standard of living; deglobalization will likely involve painful costs and may limit real income growth during the recovery.
'''

In [74]:
context = foreign_trade

topic = 'How will changes in international trade affect the economy in 2023?'

prompt = f'''Context:
{context}

Act as an expert financial analyst. Based on the given context, give an extremely detailed analysis for the following question:
{topic}'''
logger.info(f"\u001b[4mPrompt\u001b[0m\n{prompt}")

Prompt
Context:
Foreign trade
The Russian invasion of Ukraine has created some headwinds for US exporters. Lower demand from Europe (market for 15% of US exports) and a higher dollar because of greater global risk have created some short-term challenges.

Beyond the Ukraine crisis, things look more positive. Real US exports are now slightly above the prepandemic level. As global financial and economic conditions normalize, US exporters are expected to see more opportunities. Demand for US goods is likely to rise in the medium term as the global economy recovers from the pandemic and the Ukraine invasion shock. Further, more normal financial conditions would create more opportunities for investment outside the United States and less desire to hold dollars to avoid risk. That will potentially lower the dollar, making US exports more competitive globally.

Meanwhile, real US imports have fallen in the last two quarters. There has been a substantial decline in imports of consumer goods, re

In [ ]:
payload = {"prompt": prompt, "maxTokens": 500, "temperature": 0}
response = query_endpoint_with_json_payload(url, payload)

logger.info(f'{response}')


Consumer spending is a key driver of economic growth, accounting for about 70% of GDP in the United States. As the economy recovers from the COVID-19 pandemic, consumer spending is expected to play a key role in driving economic growth. However, consumer spending trends are expected to vary significantly in 2023, depending on several factors.

One key factor that will shape consumer spending trends in 2023 is the level of savings that households have accumulated during the pandemic. During the pandemic, households saved a significant amount of money, as spending on consumer services fell and spending on durable goods increased. As households begin to spend down their savings, consumer spending is expected to rise. However, the level of savings that households have accumulated and their willingness to spend it will determine the pace of consumer spending growth in 2023.

Another factor that will shape consumer spending trends in 2023 is the recovery of consumer services. As the economy

In [196]:
# Some prompt templates needed for the Prompt Chaining simulation below
stored_documents = consumer_spending + '\n' + foreign_trade

retrieval_prompt = '''Context:
{stored_documents}

Act as an expert financial analyst. Use insights based on the given context, give an extremely detailed analysis for the following topic:
{question}'''

report_writing_prompt = '''You are an expert financial analysis. Based on the provided information below, you can draw insights and write an extremely detailed analysis for any given report.
In the report, you are to make intelligent financial judgements and tell your readers your own opinions based on the provided information.

Information:
{information}


Write a report for {report}:
'''

### Prompt Chaining
This section explores how we can chain all the previous steps together to generate a final report using all the prompt templates we have defined above. It simulates the chaining of Auto Prompt Generation -> Query documents -> Report writing

Note: Ensure that all previous cells in this notebook have been executed before running the cell below.

In [205]:
# Report to be written
report = "2023 financial outlook for United States"
generate_analysis_prompt.format(report=report)

# Auto Prompt Generation
generate_analysis_questions_payload = {"prompt": generate_analysis_prompt, "maxTokens": 200, "temperature": 0.2}
analysis_questions = query_endpoint_with_json_payload(url, generate_analysis_questions_payload)

logger.info(f'*Analysis Questions:*\n{analysis_questions}')

# Steps to convert the questions into a list and remove the numbering
questions = analysis_questions.split('\n')
for i in range(len(questions)):
    questions[i] = questions[i][3:]

extracted_information = ''
# For each questions, retrieve the answer from the 'stored documents' and store them in a single variable as extracted_information
for q in questions:
    retrieval = retrieval_prompt.format(stored_documents=stored_documents, question=q)
    retrieval_payload = {"prompt": retrieval, "maxTokens": 500, "temperature": 0.2}
    info = query_endpoint_with_json_payload(url, retrieval_payload)
    extracted_information = extracted_information + info + '\n'
    logger.info(f'\n\u001b[4mExtracted Informations for: {q}\u001b[0m{info}')


# Use the extracted information to write the final report
report_writing_prompt = report_writing_prompt.format(information=extracted_information, report=report)
report_writing_payload = {"prompt": report_writing_prompt, "maxTokens": 700, "temperature": 0.9}
final_report = query_endpoint_with_json_payload(url, report_writing_payload)

logger.info(f'\n\u001b[4mFinal Report\u001b[0m\n{final_report}')

*Analysis Questions:*
1. How will consumer spending in the United States affect the country's financial outlook in 2023?
2. How will foreign trade with the United States affect the country's financial outlook in 2023?

Extracted Informations for: How will consumer spending in the United States affect the country's financial outlook in 2023?
Consumer spending in the United States is expected to remain strong in 2023, driven by continued job and income growth. However, spending will slow as the savings rate eventually rises back to the 6% range.

The invasion of Ukraine by Russia has created some headwinds for US exporters, but demand for US goods is likely to rise in the medium term as the global economy recovers from the pandemic and the Ukraine invasion shock.

The shift toward deglobalization is likely to continue, and this will create some challenges for US exporters. Building more robust supply chains may mean moving production back to the United States, or perhaps to Mexico or som